In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pickle
import requests
import numpy as np
import pandas as pd
import json
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0'}

In [2]:
search_add = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=5&kw=physics&ol=false&z='
zip_code = 78750
url = search_add + str(zip_code)

chromedriver = '/home/codonnell/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

#run this, then physically enter fields to get to search page
#then reopen search url above and it should work this time
#search for all tutor links on page

In [3]:
def get_tutor_info(zip_code, links, headers):
    
    mains = []
    scheds = []
    empty = {'sunday': [None], 'monday': [None], 'tuesday': [None], 'wednesday': [None],
         'thursday': [None], 'friday': [None], 'saturday': [None]}
    
    for i, link in enumerate(links):
        r = requests.get(link, headers = headers).text
        soup = BeautifulSoup(r, 'html.parser')

        tags = soup.find_all('script')
        try:
            time_tutoring = soup.find_all('h3')[0].text
        except:
            time_tutoring = []
        bio_list = soup.find_all('p', {'class':  'spc-zero-n spc-sm-s'})
        bio = [part.text for part in bio_list]
        
        if soup.find_all('i' ,{'class':'wc-background-pass wc-green'}):
            bck = 1
            bck_passed = soup.find_all('p', {'class':"spc-zero"})[1].text
        else:
            bck = 0
            bck_passed = []
            
        edu = soup.find_all('section' ,{'class':"spc-med-s"})
        if edu:
            educ =[e.text for e in edu]
        else:
            educ = []
        
        main_details = json.loads(tags[0].string)
        main_details['time_tutoring'] = time_tutoring
        main_details['bio'] = bio
        main_details['background_check'] = bck
        main_details['date_background_passed'] = bck_passed
        main_details['education'] = educ
        
        mains.append(main_details)
        try:
            scheds.append(json.loads(tags[6].string.split('=')[1][:-1]))
        except:
            scheds.append(empty)
        time.sleep(np.random.rand()*5)
        
    df = pd.merge(pd.DataFrame(mains), pd.DataFrame(scheds), left_index=True, right_index=True)
    df.to_csv('/home/codonnell/tutors/tutnew/tutors_' + str(zip_code) + '.csv')

In [4]:
def save_tut_id(unique_tuts):
    with open('/home/codonnell/tutors_ids.txt', 'w') as filehandle:
        for item in unique_tuts:
            filehandle.write('%s\n' % item)

In [10]:
already_done = [77449,11368,60629,90011,79936,30044,80209,89138,28208,84102,98122,33128,
               46201, 55401,43215,19102,94108,22046,91911,37128,85001,78214,48201,63108,
               80303, 926, 8701]
pop_zips = pd.read_csv('/home/codonnell/Insight/Insight_Project_AsTutor/data/raw/zips_incomes/zips_il.csv')

np.setdiff1d(pop_zips['zip_code'].values,already_done)

array([60004, 60016, 60056, 60073, 60085, 60120, 60148, 60402, 60411,
       60440, 60453, 60505, 60506, 60608, 60609, 60614, 60617, 60618,
       60619, 60620, 60623, 60625, 60628, 60630, 60632, 60634, 60638,
       60639, 60640, 60641, 60643, 60647, 60651, 60657, 60804])

In [11]:
zips = np.setdiff1d(pop_zips['zip_code'].values,already_done)[-7:]
#zips = already_done
subjects = ['physics', 'math', 'spanish', 'english', 'computer']
links = []
tot_tuts = []

for zip_code in zips:
    links = []
    tot_tuts = []
    
    for subject in subjects:
            
        url = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
        driver.get(url)
        
        try:
            num_tuts = driver.find_elements_by_tag_name('strong')[2].text #per subject given zip
        except:
            num_tuts = 0
        
        lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
        lks_found = [link.get_attribute('href') for link in lks]
        links.append(lks_found)
        
        try:
            if int(num_tuts.split(' ')[0]) > 10:
                extras = int(np.floor(int(num_tuts.split(' ')[0])/10)) + 1
                for i in range(1,extras):
                    urlmore = 'https://www.wyzant.com/match/search/more?d=20&gender_pref=none&kw=' + subject + '&max_age=100&max_price=500&min_age=18&min_price=10&ol=false&page_number=' + str(i) + '&sort=1&st=0&utc_offset=-5&z=' +str(zip_code)
                    driver.get(urlmore)
                    more_lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
                    more_lks_found = [link.get_attribute('href') for link in more_lks]
                    links.append(more_lks_found)
                    time.sleep(np.random.rand()*5)
            else:
                pass
        except:
            pass
            
            
        tot_tuts.append(num_tuts) #per subject given zip
            
            
        time.sleep(np.random.rand()*5)
    
    unique_tuts = np.unique(np.array([item for sublist in links for item in sublist]))
    save_tut_id(unique_tuts)
    get_tutor_info(zip_code, unique_tuts, headers)
    tut_num_df = pd.DataFrame({'zip_code': [str(zip_code)]*5, 'tot_tuts': tot_tuts})
    tut_num_df.to_csv('/home/codonnell/tutors//tutnew/tut_num_' +str(zip_code) + '.csv')
    
    time.sleep(np.random.rand()*5)

In [7]:
#driver.get(url)
num_tuts = driver.find_elements_by_tag_name('strong')[2].text #per subject given zip

In [16]:
[i for i in range(1,2)]

[1]